Execute the following cells in python

In [ ]:
import pandas as pd
import warnings
import ast
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("/content/ALS_Pubtator_20a.csv",encoding = "latin-1")

In [ ]:
df['Abstracts'] = df['Abstracts'].astype(str)
with open('pathways.txt', 'r') as file:
    words_to_search = [word.strip().lower() for word in file.readlines()]
matches = {}
for index, row in df.iterrows():
    pmid = row['PMID']
    paragraph = row['Abstracts'].lower()  # Convert paragraph to lowercase for case-insensitive matching
    matches[pmid] = []
    for word in words_to_search:
        if word in paragraph:
            matches[pmid].append(word)
matches_df = pd.DataFrame({'pmid': list(matches.keys()), 'pathways': list(matches.values())})

In [ ]:
matches_df.to_csv("ALS_Pathways_20.csv", index=False)

In [ ]:
unique_matched_words = matches_df['pathways'].explode().unique()

# Print the unique matched words
print(len(unique_matched_words))

144


In [ ]:
pathways_df = pd.read_csv("/content/ALS_Pathways_20.csv")

In [ ]:
for index,row in pathways_df.iterrows():
  pathways_df['pathways'][index] = ast.literal_eval(pathways_df['pathways'][index])
empty_list_count = pathways_df['pathways'].apply(lambda x: len(x) == 2).sum()
print("Number of rows with empty lists:", empty_list_count)
df_exploded = pathways_df['pathways'].explode()
unique_pathways = df_exploded.unique()
print(unique_pathways)

In [ ]:
import numpy as np

# Assuming unique_pathways is a numpy array
unique_pathways_list = unique_pathways.astype(str)

# Replace nan values with empty strings
unique_pathways_list[unique_pathways_list == 'nan'] = ''

# Convert to a list and remove duplicates
unique_pathways_list = list(set(unique_pathways_list))

# Remove empty string from the list if needed
unique_pathways_list = [pathway for pathway in unique_pathways_list if pathway]

unique_count = len(set(unique_pathways_list))
set_of_pathways  = set(unique_pathways_list)

In [ ]:
pathways_df = pd.read_csv("/content/ALS_Pathways_20.csv")

In [ ]:
import pandas as pd

# Assuming set_of_pathways is a set containing pathway names

pmid_series = []
pathways_series = []
length_series = []

for pathway in set_of_pathways:
    filter_df = pathways_df[pathways_df['pathways'].str.contains(pathway)]
    pmids = filter_df['pmid'].tolist()
    pmidlst = ""
    for p in pmids:
      #print(p)
      pmidlst = pmidlst+ str(p)+","
    pmidlst = pmidlst[:-1]
    #print(pmidlst)
    pathways_series.append(pathway)
    pmid_series.append(pmidlst)
    length_series.append(len(pmids))


In [ ]:
consensus_pathways_df = pd.DataFrame({'pathways': pathways_series, 'PMID': pmid_series,'length':length_series})
consensus_pathways_df.to_csv("ALS_Pathways_20_final.csv", index=False)

Download the above file. This is the final pathways file. Next steps to generate pathway sentences.

In [ ]:

ids_Data = pd.read_csv('/content/PMID_PMCID_step.csv')
column_names4 = ["Pathways", "PMID", "PMCID"]
my_dataframe4 = pd.DataFrame(columns=column_names4)
for i in range(len(consensus_pathways_df)):
  g = consensus_pathways_df['pathways'][i]
  pmids = consensus_pathways_df['PMID'][i]
  pmids = pmids.split(",")
  for pid in pmids:
    #print(pid)
    p = float(pid)
    #print(p)
    df1_index = ids_Data.index[ids_Data['PMID'] == int(p)]
    pmcid = ids_Data._get_value(df1_index[0],'PMCID')
    my_dataframe4 = my_dataframe4.append({"Pathways": g, "PMID":pid , "PMCID": pmcid}, ignore_index=True)

In [ ]:
my_dataframe4['Sentence'] = ''
import re
for i in range(len(my_dataframe4)):
  g = my_dataframe4['Pathways'][i]
  pmid = my_dataframe4['PMID'][i]
  p = float(pmid)
  pmcid = my_dataframe4['PMCID'][i]
  if str(pmcid) == "nan":
    filtered_df = df[df['PMID'] == int(p)]
    abs = filtered_df['Abstracts']
    split_lines = abs.tolist()
    str1 = split_lines[0]
    split_lines = str1.split(".")
    lines = ""
    for line in split_lines:
      line = re.sub(r'\n', '', line)
      line = line.lower()
      g = g.lower()
      if g in line:
        lines = lines + line +","
    my_dataframe4['Sentence'][i] = str(lines)

In [ ]:
my_dataframe4.to_csv('ALS_Path_Sen_20.csv')

Download the above file and run the following cells in R

In [ ]:
install.packages('pubmed.mineR')
library(stringr)
library(pubmed.mineR)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
path_df <- read.csv('/content/ALS_Path_Sen_20.csv')
path_df[path_df == ""] = "nan"
nrow(path_df)

[1] 19007

In [ ]:
for(i in 17000:nrow(path_df))
{
  g <- path_df[i,'Pathways']
  pmcid <- path_df[i,'PMCID']
  if(pmcid == "nan")
  {
  }
  else
  {
    x <- path_df[i,'Sentence']
    if(x == "nan")
    {
    modified_id <- sub("PMC", "", pmcid)
    modified_id <- as.integer(modified_id)
    #print(modified_id)
    sentence = Give_Sentences_PMC(modified_id,g)
    result <- paste(sentence, collapse = ", ")
    path_df[i,'Sentence']=result
    }
  }
}

In [ ]:
write.csv(path_df,'ALS_Path_Sentence_20.csv')

Interaction - Gene to pathways, pathways to disease: To execute in python

In [ ]:
import pandas as pd
import re

# Load the Chemicals and Genes CSV files into dataframes
diseases_df = pd.read_csv('/content/ALS_Disease_Sen_20.csv',encoding = "latin-1")
genes_df = pd.read_csv('/content/ALS_Genes_Sen_20.csv',encoding = "latin-1")
path_df = pd.read_csv('/content/ALS_Path_Sentence_20.csv',encoding = "latin-1")

In [ ]:
cols = ['Diseases', 'PMID', 'PMCID', 'Sentence']
diseases_df = diseases_df[cols]
cols1 = ['Genes', 'PMID', 'PMCID', 'Sentence']
genes_df = genes_df[cols1]
cols2 = ['Pathways', 'PMID', 'PMCID', 'Sentence']
path_df = path_df[cols2]

In [ ]:
# Initialize lists to store the extracted data
pmid_list = []
pmcid_list = []
sentence_list = []
genes_list = []
pathways_list = []
interaction_type_list = []
regulation_list = []

# Define regular expressions for different interaction types
interaction_patterns = {
    'Inhibition': r'\b(inhibit(?:s|ing)?|inhibition)\b',
    'Activation': r'\b(activate(?:s|ing)?|activation)\b',
    'Proliferation': r'\bproliferation\b',
    'Allosteric': r'\ballosteric\b',
    'Agonist': r'\bagonist\b',
    'Antagonist': r'\bantagonist\b'
}

# Iterate through each row in the Chemicals dataframe
for path_row in path_df.itertuples():
    for gene_row in genes_df.itertuples():
        # Check if the PMID and PMCID match in both dataframes
        if str(path_row.PMID) == str(gene_row.PMID) and str(path_row.PMCID) == str(gene_row.PMCID):
            sentence = str(path_row.Sentence)  # Ensure sentence is a string
            pathways = str(path_row.Pathways)  # Convert to string
            genes = str(gene_row.Genes)  # Convert to string

            # Initialize interaction type and regulation as 'Other'
            interaction_type = 'Other'
            regulation = 'Other'

            # Check for interaction type using regular expressions
            for key, pattern in interaction_patterns.items():
                if re.search(pattern, sentence, re.IGNORECASE):
                    interaction_type = key
                    break

            # Use regular expressions to identify regulation
            if re.search(r'\b(up(?:-| )?regulated)\b', sentence, re.IGNORECASE):
                regulation = 'Up'
            elif re.search(r'\b(down(?:-| )?regulated)\b', sentence, re.IGNORECASE):
                regulation = 'Down'
            print(str(path_row.PMID)+" "+str(path_row.PMCID)+" "+genes+" "+pathways)
            # Append the extracted data to respective lists
            pmid_list.append(str(path_row.PMID))
            pmcid_list.append(str(path_row.PMCID))
            sentence_list.append(sentence)
            genes_list.append(genes)
            pathways_list.append(pathways)
            interaction_type_list.append(interaction_type)
            regulation_list.append(regulation)

# Create a new dataframe with the extracted data
result_df = pd.DataFrame({
    'PMID': pmid_list,
    'PMCID': pmcid_list,
    'Sentence': sentence_list,
    'Genes': genes_list,
    'Pathways': pathways_list,
    'Interaction type': interaction_type_list,
    'Regulation': regulation_list
})

# Save the new dataframe to a CSV file
result_df.to_csv('interaction_genepath_results.csv', index=False)



In [ ]:
pmid_list = []
pmcid_list = []
sentence_list = []
pathways_list = []
diseases_list = []
interaction_type_list = []
regulation_list = []

# Define regular expressions for different interaction types
interaction_patterns = {
    'Inhibition': r'\b(inhibit(?:s|ing)?|inhibition)\b',
    'Activation': r'\b(activate(?:s|ing)?|activation)\b',
    'Proliferation': r'\bproliferation\b',
    'Allosteric': r'\ballosteric\b',
    'Agonist': r'\bagonist\b',
    'Antagonist': r'\bantagonist\b'
}

# Iterate through each row in the Chemicals dataframe
for dis_row in diseases_df.itertuples():
    for path_row in path_df.itertuples():
        # Check if the PMID and PMCID match in both dataframes
        if str(dis_row.PMID) == str(path_row.PMID) and str(path_row.PMCID) == str(path_row.PMCID):
            sentence = str(dis_row.Sentence)  # Ensure sentence is a string
            diseases = str(dis_row.Diseases)  # Convert to string
            pathways = str(path_row.Pathways)  # Convert to string

            # Initialize interaction type and regulation as 'Other'
            interaction_type = 'Other'
            regulation = 'Other'

            # Check for interaction type using regular expressions
            for key, pattern in interaction_patterns.items():
                if re.search(pattern, sentence, re.IGNORECASE):
                    interaction_type = key
                    break

            # Use regular expressions to identify regulation
            if re.search(r'\b(up(?:-| )?regulated)\b', sentence, re.IGNORECASE):
                regulation = 'Up'
            elif re.search(r'\b(down(?:-| )?regulated)\b', sentence, re.IGNORECASE):
                regulation = 'Down'

            # Append the extracted data to respective lists
            pmid_list.append(str(dis_row.PMID))
            pmcid_list.append(str(dis_row.PMCID))
            sentence_list.append(sentence)
            pathways_list.append(pathways)
            diseases_list.append(diseases)
            interaction_type_list.append(interaction_type)
            regulation_list.append(regulation)

# Create a new dataframe with the extracted data
result_df = pd.DataFrame({
    'PMID': pmid_list,
    'PMCID': pmcid_list,
    'Sentence': sentence_list,
    'Pathways': pathways_list,
    'Diseases': diseases_list,
    'Interaction type': interaction_type_list,
    'Regulation': regulation_list
})

# Save the new dataframe to a CSV file
result_df.to_csv('interaction_pathdis_results.csv')

Save the above files in the system

---


Summary Generation for pathways in python

In [ ]:
!pip install bert-extractive-summarizer
from summarizer import Summarizer
model = Summarizer()

In [ ]:
import pandas as pd
data4 = pd.read_csv('/content/ALS_Path_Sentence_20.csv',encoding = "latin-1")

In [ ]:
pathways = data4['Pathways']
pathways = set(pathways)
len(pathways)

143

In [ ]:
path_summary = pd.DataFrame(columns=['Pathways', 'Summary'])
usp = []
i = 1
for c in pathways:
  text = ""
  for i in range(len(data4)):
    str2 = data4['Pathways'][i]
    if(c.casefold() == str2.casefold()):
      if(isinstance(data4['Sentence'][i],str)):
        text = text + data4['Sentence'][i]
  text = text.replace(",",".")
  print(len(text))
  print(i)
  i =i+1
  if(len(text)>1000000):
    usp.append(c)
  else:
    result = model(text, ratio = 0.15)
    row = {'Pathways':c,'Summary':result}
    path_summary = path_summary.append(row,ignore_index = True)

In [ ]:
usp

['amyotrophic lateral sclerosis']

In [ ]:
path_summary.to_csv('Pathways_summary.csv')